In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import os

In [2]:
# df_china = pd.read_csv('output/merged_china.csv')
# df_us = pd.read_csv('output/merged_us.csv')
# df_india = pd.read_csv('output/merged_india.csv')

# df_raw_owid = pd.read_csv('dataset/owid/owid-co2-data.csv')
# df_raw_ghg = pd.read_csv('dataset/owid/ghg-emissions-by-sector.csv')
# df_raw_worldbank = pd.read_csv('dataset/worldbank/API.csv')
# df_worldbank_meta_country = pd.read_csv('dataset/worldbank/Metadata_Country_API_19_DS2_en_csv_v2_3159902.csv')


df_worldbank_imputed = pd.read_csv('output/dataset_worldbank_imputed.csv')

## Overall process

### Preprocessing (again)
1. Identify null value
2. Overlapping columns (or indicators)
2. Imputation 
    - woid (imputation drop columns)
    - worldbank (imputation
3. Data transformation

reference: [5 stages of data prep for k-means](https://medium.com/@evgen.ryzhkov/5-stages-of-data-preprocessing-for-k-means-clustering-b755426f9932)

### EDA
- Only looking at a few attributes

#### For OWID
- test

#### For Worldbank
- Test


### K-Means
- Euclidean K Means
- DBA K Means
- Soft DTW K Means

## EDA

In [3]:
df_worldbank_imputed

,Country Name,Year,Urban population (% of total population),Population growth (annual %),Methane emissions (kt of CO2 equivalent),CO2 intensity (kg per kg of oil equivalent energy use),Energy use (kg of oil equivalent per capita)
0,Aruba,1960,50.776,2.236858,283583.062664,2.863799,1914.972357
1,Aruba,1961,50.761,2.236462,283586.955510,2.845710,1913.773722
2,Aruba,1962,50.746,1.432843,284129.953488,2.773063,1738.412785
3,Aruba,1963,50.730,0.823502,284542.777005,2.713592,1605.106452
4,Aruba,1964,50.715,0.580334,284709.618454,2.679013,1551.267704
...,...,...,...,...,...,...,...
15550,Zimbabwe,2016,32.296,1.549294,11380.000000,1.421311,438.447490
15551,Zimbabwe,2017,32.237,1.459406,11560.000000,1.396260,414.750358
15552,Zimbabwe,2018,32.209,1.410382,11850.000000,1.374628,402.110531
15553,Zimbabwe,2019,32.210,1.421142,288207.742939,1.383934,425.865325


## Clustering

### Preprocessing

Missing value is being handled by imputation. This preprocessing is to pivot (or transpose) data in column-year format (columns of years), in order to fit into tslearn's kmeans clustering.

In [54]:
# function to slice out country from the df
# and transpose the data to become column-year format
def get_pivot_data_column_year(df, country_name):
    
    # slice out targeted country and store it inside df_country
    df_country = df[df['Country Name'] == f'{country_name}'].copy()
    df_country.reset_index(inplace=True, drop=True) 

    # get ready to transpose
    df_country = df_country.iloc[:, 1:] # remove 'Country Name'
    df_country.Year = df_country.Year.astype('str') # convert 'year' to string type
    df_country = df_country.transpose()
    df_country.reset_index(inplace=True) 

    # reset first row as column
    new_header = df_country.iloc[0] # grab the first row for the header
    df_country = df_country[1:] # take the data but not header
    df_country.columns = new_header # set the header row as the df header
    df_country.rename(columns={'Year': 'Indicator Name'}, inplace=True) # rename the column column
    df_country.reset_index(inplace=True, drop=True) 

    # adding new columns
    df_country['Country Name'] = f'{country_name}'

    # rearrange columns
    col = df_country.columns.tolist()
    new_col = col[-1:] + col[:-1]
    
    return df_country[new_col]

In [59]:
# define countries, years and col
countries = df_worldbank_imputed['Country Name'].unique().tolist()
years = df_worldbank_imputed['Year'].unique()
years = years.tolist()
years = [str(year) for year in years]
columns = ['Country Name', 'Indicator Name'] + years

df_worldbank_transposed = pd.DataFrame([], columns=columns)
for country in countries:
    df_temp = get_pivot_data_column_year(df_worldbank_imputed, country) 
    df_worldbank_transposed = pd.concat([df_worldbank_transposed, df_temp], axis=0, ignore_index=True)
    
df_worldbank_transposed

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,Urban population (% of total population),50.776,50.761,50.746,50.73,50.715,50.7,50.685,50.67,...,42.94,42.957,42.99,43.041,43.108,43.192,43.293,43.411,43.546,43.697
1,Aruba,Population growth (annual %),2.23686,2.23646,1.43284,0.823502,0.580334,0.573498,0.599694,0.590951,...,0.377979,0.503385,0.58329,0.590508,0.541048,0.50286,0.471874,0.459266,0.437415,0.428017
2,Aruba,Methane emissions (kt of CO2 equivalent),283583,283587,284130,284543,284710,284718,284704,284713,...,286561,286473,286413,286397,286416,286424,286423,286406,286391,286365
3,Aruba,CO2 intensity (kg per kg of oil equivalent ene...,2.8638,2.84571,2.77306,2.71359,2.67901,2.66049,2.6442,2.62555,...,1.67288,1.66399,1.65233,1.63609,1.61634,1.59769,1.57988,1.56366,1.54715,1.53182
4,Aruba,Energy use (kg of oil equivalent per capita),1914.97,1913.77,1738.41,1605.11,1551.27,1548.67,1553.24,1550.23,...,956.627,984.962,1004.56,1009.66,1003.61,1001.2,1001.54,1007.07,1011.8,1020.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1270,Zimbabwe,Urban population (% of total population),12.608,12.821,13.082,13.578,14.092,14.62,15.165,15.727,...,33.015,32.834,32.654,32.504,32.385,32.296,32.237,32.209,32.21,32.242
1271,Zimbabwe,Population growth (annual %),2.59288,3.34225,3.37814,3.39575,3.39094,3.37361,3.34655,3.32865,...,1.53641,1.69808,1.77767,1.75474,1.66369,1.54929,1.45941,1.41038,1.42114,1.47387
1272,Zimbabwe,Methane emissions (kt of CO2 equivalent),291683,291133,291053,290933,290824,290721,290620,290510,...,11820,11730,11700,10820,11790,11380,11560,11850,288208,288166
1273,Zimbabwe,CO2 intensity (kg per kg of oil equivalent ene...,2.11346,2.15091,2.14089,2.13439,2.12674,2.11852,2.10998,2.10241,...,1.12079,1.12025,1.09517,1.47497,1.44869,1.42131,1.39626,1.37463,1.38393,1.37041


### Time Series K Means

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

# Matplotlib customization
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams['font.size'] = 14
mpl.rcParams['figure.dpi'] = 150.
mpl.rcParams["figure.figsize"] = (20,50)

In [ ]:
seed = 1
np.random.seed(seed)

In [ ]:
# Set number of cluster

cluster_number = 10

In [ ]:
# training set (there's no testing set)

X_train_co2 = to_time_series_dataset(df_co2_normalized.copy())

In [ ]:
def euclideanKMeans(cluster, seed, X_train):
    print("Euclidean k-means")
    km = TimeSeriesKMeans(n_clusters=cluster, 
                          verbose=True, 
                          random_state=seed, 
                          max_iter=10)
    y_pred = km.fit_predict(X_train)
#     clusters = pd.Series(data=y_pred, index=X_train.index)
#     clusters

    plt.figure()
    for yi in range(cluster):
        plt.subplot(cluster, 1, yi+1)
        for xx in X_train[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2)
        plt.plot(km.cluster_centers_[yi].ravel(), "r-")
        plt.ylim(0, 1)
        plt.text(0.01, 0.50,'Cluster %d' % (yi + 1),
                 transform=plt.gca().transAxes)

    print("Euclidean k-means Chart")
    plt.show()
    return y_pred

In [ ]:
# DBA-k-means
def dbaKMeans(cluster, seed, X_train):
    print("DBA k-means")
    dba_km = TimeSeriesKMeans(n_clusters=cluster,
                              n_init=2,
                              metric="dtw",
                              verbose=True,
                              max_iter_barycenter=10,
                              random_state=seed)
    y_pred = dba_km.fit_predict(X_train)

    for yi in range(cluster):
        plt.subplot(cluster, 1, yi+1)
        for xx in X_train[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2)
        plt.plot(dba_km.cluster_centers_[yi].ravel(), "r-")
        plt.ylim(0, 1)
        plt.text(0.01, 0.50,'Cluster %d' % (yi + 1),
                 transform=plt.gca().transAxes)


    print("DBA k-means Chart")
    plt.show()
    return y_pred

In [ ]:
# Soft-DTW-k-means
def softDTWKmean(cluster, seed, X_train):
    print("Soft-DTW k-means")
    sdtw_km = TimeSeriesKMeans(n_clusters=cluster,
                               metric="softdtw",
                               metric_params={"gamma": .01},
                               verbose=True,
                               random_state=seed)
    y_pred = sdtw_km.fit_predict(X_train)

    for yi in range(cluster):
        plt.subplot(cluster, 1, yi+1)
        for xx in X_train[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2)
        plt.plot(sdtw_km.cluster_centers_[yi].ravel(), "r-")
        plt.ylim(0, 1)
        plt.text(0.01, 0.50,'Cluster %d' % (yi),
                 transform=plt.gca().transAxes)

    print("Soft-DTW k-means Chart")
    plt.show()
    return y_pred

In [ ]:
def mergeClusterNames(y_pred, df_index):
    clusters = pd.Series(data=y_pred, index=df_index.index)
    df_cluster = clusters.to_frame()
    df_cluster.columns = ['cluster']
    return df_cluster

def getSingleCluster(df_cluster, n):
    # cluster 1 in the chart represent cluster 0 in the data.
    display(df_cluster[df_cluster['cluster'] == n-1])

In [ ]:
y_pred_X_china_euclideanKM = euclideanKMeans(15, seed, X_train_co2)

In [ ]:
cluster_china_euclideanKM = mergeClusterNames(y_pred_X_china_euclideanKM, df_china_normalized_tranposed)
getSingleCluster(cluster_china_euclideanKM, 2)